In [1]:
# fak_NUMER = r'data i miejsce wystawienia\n\n(.*)\n'
# fak_DATA_WYSTAWIENIA = r'(\d{4}-\d{2}-\d{2}) .*\ndata i miejsce wystawienia'
# fak_KWOTY = r'W tym.*?Brutto(.*?)Razem do'
# fak_REZERWACJA = r'Prowizja .* (1\d{6})'

In [ ]:
import pdfminer
import pdfminer.high_level
import os
import pandas as pd

In [2]:
if not os.path.exists('done_2503'):
    os.makedirs('done_2503')

In [3]:
num_pages = 0
texts = []
for file in os.listdir('.'):
    if file.endswith('.pdf'):
        with open(file, 'rb') as f:
            texts.append(pdfminer.high_level.extract_text(f))
        os.rename(file, 'done_2503/' + file)

In [4]:
# split all texts[] into separate columns inside a dataframe, based on he newline
df = pd.DataFrame(texts, columns=['text'])
df = df.text.str.split('\n', expand=True)

In [5]:
# print dataframe of row 0 and column 34
a = df.iloc[1, 35]

In [6]:
# if a value in column number 35 is empty, then split the value from column 34 into 2 columns and move the second one to 35. then move the rest of the columns to the right
for index, row in df.iterrows():
    if row[35] == a:
        split_values = str(row[34]).split(' ', 1)
        if len(split_values) == 2:
            df.at[index, 35] = split_values[1]
            for col in range(len(df.columns), 35, -1):
                df.at[index, col] = df.at[index, col - 1]
            df.at[index, 34] = split_values[0]

In [7]:
df

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,:,,:,,:,,:,,,None
1,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,:,,:,,:,,:,,,None
2,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,:,,:,,:,,:,,,None
3,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,:,,:,,:,,:,,,None
4,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,:,,:,,:,,:,,,None
5,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,,:,,:,,:,,,None,None
6,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,,:,,:,,:,,,None,None
7,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,,:,,:,,:,,,None,None
8,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,,:,,:,,:,,,None,None
9,SATIŞ/SALE,,SUNEXPRESS,Adres/Address:,,Güneş Ekspres Havacılık A.Ş,"Yenigöl Mahallesi, Nergiz Sokak",No: 84 PK No: 07230,Muratpaşa / Antalya Türkiye,,...,,:,,:,,:,,:,,


In [8]:
# drop na columns
df = df.dropna(axis=1, how='all')

In [9]:
# drop column where the only values are '', ' '
df = df.drop(df.columns[(df == '').all()], axis=1)
df = df.drop(df.columns[(df == ' ').all()], axis=1)
df = df.drop(df.columns[(df == a).all()], axis=1)

In [10]:
# remove illigal characters
df = df.replace(to_replace=r'\f', value='', regex=True)

In [11]:
# save to excel
df.to_excel('output.xlsx', index=False)

In [13]:
# dane = [['Mersis No', 'Tax Administration', 'Passenger Name', 'SunExoress Ref No', 'Document No', 'Invoice Date', 'Fee', 'Tax', 'Total']]

# for i, page in enumerate(pages):
#     if i > 0:
#         try:
#             if file_names[i] != file_names[i-1]:
#                 current_page_number = 1
#             else:
#                 current_page_number += 1
#         except IndexError:
#             if i < num_pages:
#                 print('IndexError at', i, ', file ', file_names[i-1], 'page', current_page_number)
#     try:
#         numer = re.search(fak_NUMER, page).group(1)
#         data_wystawienia = re.search(fak_DATA_WYSTAWIENIA, page).group(1)
#         rezerwacja = re.search(fak_REZERWACJA, page).group(1)
#         kwoty = re.search(fak_KWOTY, page, re.DOTALL).group(1).replace(' ', '').replace('\n', ' ').replace('%', '% ').strip()
#         kw2 = [kwota for kwota in kwoty.split(' ') if kwota != '']
#         kwota_netto, stawka_vat, kwota_vat, kwota_brutto = kw2
#         faktura = numer, data_wystawienia, kwota_netto, stawka_vat, kwota_brutto, rezerwacja, file_names[i], current_page_number
#         if rezerwacja not in [item[5] for item in faktury]:
#             faktury.append(faktura)
#     except AttributeError:
#         if i != num_pages:
#             print(i+1, 'error')
#             faktury.append(['n/a', 'n/a', 'n/a', 'n/a', 'n/a', 'n/a', file_names[i], current_page_number])
#     except ValueError:
#         faktury.append(['check', 'check', 'check', 'check', 'check', 'check', file_names[i], current_page_number])
#         print('you need to check ' + file_names[i], 'page ' + str(current_page_number))

# df = pandas.DataFrame(faktury)
# df.to_excel('wakacje_excel.xlsx', index=False, header=False)